In [3]:
!pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 5.2 MB/s 


In [4]:
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.layers import Conv2D, MaxPool2D, BatchNormalization, Flatten, Dense, Activation, Dropout, Input
from keras.models import Model
from tensorflow_addons.layers import SpatialPyramidPooling2D
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split



from keras.datasets import cifar10
from keras.utils import np_utils
import matplotlib.pyplot as plt

(x_train,y_train),(x_test,y_test)=cifar10.load_data()
x_train = x_train / 255.
x_test = x_test / 255.
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)



# SPP-net
input_layer = Input((None, None, 3))
x = input_layer

x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
x = BatchNormalization()(x)
x = MaxPool2D((2, 2), strides=(2, 2), padding='same', name='block1_maxpool')(x)

x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
x = BatchNormalization()(x)

x = SpatialPyramidPooling2D([1, 3, 5])(x)

x = Flatten(name='flatten')(x)

x = Dense(units=1024, activation='relu', name='fc1')(x)
x = Dense(units=10, activation='softmax', name='predictions')(x)

SPP_net = Model(inputs=input_layer, outputs=x)
SPP_net.compile(loss='categorical_crossentropy', metrics=['acc'])

SPP_net.summary()

170498071/170498071 [==============================] - 3s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 batch_normalization (BatchN  (None, None, None, 64)   256       
 ormalization)                                                   
                                                                 
 block1_maxpool (MaxPooling2  (None, None, None, 64)   0         
 D)                                                              
                                                            

In [5]:
def model_train(model, x_train, y_train, batch_size, k, shuffle, epochs):
    # kerasで作成したモデルをk分割交差検証を使ってトレーニングする関数
    kf = KFold(n_splits=k, shuffle=shuffle)

    epoch = 0
    loss_list = []
    val_loss_list = []
    acc_list = []
    val_acc_list = []

    for _ in range(int(epochs/k)):
        ep_loss, ep_val_loss, ep_acc, ap_val_acc = 0, 0, 0, 0  # 1エポックごとの各値を保持する変数を初期化
        for train_index, val_index in kf.split(x_train, y_train):
            epoch += 1
            print(f"Epoch: {epoch}")
            history=model.fit(x_train[train_index], y_train[train_index], 
                              batch_size=batch_size, 
                              validation_data=(x_train[val_index], y_train[val_index]))

            loss_list.append(history.history['loss'])
            val_loss_list.append(history.history['val_loss'])
            acc_list.append(history.history['acc'])
            val_acc_list.append(history.history['val_acc'])

    return [loss_list, val_loss_list, acc_list, val_acc_list]

def model_test(model, x_test, y_test):
    test_acc = np.mean(np.argmax(y_test, axis=1) == np.argmax(model.predict(x_test), axis=1))
    return test_acc
    
    
    
    
    # training
spp_history = model_train(model=SPP_net, x_train=x_train, y_train=y_train, batch_size=128, k=5, shuffle=True, epochs=5)



print(f"SPP_net: {model_test(SPP_net, x_test, y_test)}")

Epoch: 1
313/313 [==============================] - 785s 3s/step - loss: 1.3396 - acc: 0.5615 - val_loss: 5.1045 - val_acc: 0.1699
Epoch: 2
313/313 [==============================] - 774s 2s/step - loss: 0.7086 - acc: 0.7569 - val_loss: 1.4031 - val_acc: 0.5969
Epoch: 3
313/313 [==============================] - 776s 2s/step - loss: 0.5011 - acc: 0.8305 - val_loss: 0.8735 - val_acc: 0.7439
Epoch: 4
313/313 [==============================] - 762s 2s/step - loss: 0.3388 - acc: 0.8856 - val_loss: 0.7603 - val_acc: 0.7565
Epoch: 5
313/313 [==============================] - 47s 149ms/step
SPP_net: 0.6701


In [6]:
import io
import requests
import numpy as np
from PIL import Image
from io import BytesIO

car_url = "https://agirobots.com/wp/wp-content/uploads/2021/05/car-604019_1920.jpg"
img = np.array(Image.open(io.BytesIO(requests.get(car_url).content)))
img = img[np.newaxis]

cifar10_class_name = ["airplane (飛機)", 
                      "automobile (汽車)", 
                      "bird (鳥)", 
                      "cat (貓)", 
                      "deer (鹿)", 
                      "dog (狗)", 
                      "frog (青蛙)", 
                      "horse (馬)", 
                      "ship (船)", 
                      "truck (卡車)"]

cifar10_class_name[np.argmax(SPP_net.predict(img))]

1/1 [==============================] - 11s 11s/step


'cat (貓)'